# 03. MLflow Experiments

This notebook demonstrates how to run experiments and track them using MLflow.

In [ ]:
import sys
import os

# Add src to path
sys.path.append(os.path.abspath(os.path.join('../')))

from src.models.train_with_mlflow import main as train_main
from scripts.compare_experiments import compare_runs

## 1. Run Experiments

Running the training script will execute the following:
- Load Data
- Feature Engineering
- Hyperparameter Tuning (GridSearch) for Logistic Regression and Random Forest
- Log parameters, metrics, and artifacts to MLflow
- Uses sklearn autologging

In [ ]:
# We need to change directory to project root or ensure paths are correct.
# train_with_mlflow assumes it is run from a place where it can find 'data/raw' relative to CWD or setup.

curr_dir = os.getcwd()
if curr_dir.endswith('notebooks'):
    os.chdir('..')
    print(f"Changed directory to: {os.getcwd()}")

train_main()

## 2. Compare Experiments
Retrieve runs from MLflow and compare them.

In [ ]:
df = compare_runs()